# Зареждане на нужните библотеки

In [1]:
import numpy as np
import pandas as pd
import zipfile as zf
import json

from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Разархивиране и прочитане на данните

In [2]:
# Разархивиране на .csv файловете
with zf.ZipFile('./data/data.zip', 'r') as zip_ref:
    zip_ref.extractall('./data')

In [3]:
# Зареждане на характеристиките на филмите
movies = pd.read_csv('data/tmdb_5000_movies.csv')
movies.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [4]:
# Зареждане на актьорите и екита на филмите
credits = pd.read_csv('data/tmdb_5000_credits.csv')
credits.head(2)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


# Обработка на данните

In [5]:
# Съединяване на двете множества по заглавието на филма
df = movies.merge(credits, on='title')
df.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [6]:
# Премахване на ненужните колони
columns_picks = ['movie_id','title','overview','genres','keywords','cast','crew']
df = df[columns_picks]
df.head(2)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [7]:
# Проверка за липсващи стойности
df.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [8]:
# Премахване на липсващите стойности
df.dropna(inplace=True)

In [9]:
# Проверка за липсващи стойности
df.isnull().sum()

movie_id    0
title       0
overview    0
genres      0
keywords    0
cast        0
crew        0
dtype: int64

**Тъй като някои от колоните на DataFrame-а съдържат JSON като стойности, ще извлечем само важната информация от тях.**

In [10]:
# Създаване на фукция за извличане на стойностите на свойстава на обектите
def convert_json_data_into_array(text):
    arr=[]
    for i in json.loads(text):
        arr.append(i['name'])
    return arr

In [11]:
# Прилагане на на фукцията върху колоните 'genres' и 'keywords'
df['genres'] = df['genres'].apply(convert_json_data_into_array)
df['keywords'] = df['keywords'].apply(convert_json_data_into_array)

df.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [12]:
# Създаване на функция за извличане на първите трима главни акрьори от филма
def get_top_3_actors(cast):
    arr=[]
    counter = 0
    for i in json.loads(cast):
        if counter != 3:
            arr.append(i['name'])
            counter += 1
        else:
            break
    return arr

In [13]:
# Прилагане на функицята за актьорите върху множеството
df['cast'] = df['cast'].apply(get_top_3_actors)

df.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [14]:
# Създаване на функция за извличане на режисьорът на филма
def get_director_from_crew(crew):
    arr=[]
    for i in json.loads(crew):
        if i['job'] == 'Director':
            arr.append(i['name'])
            break
    return arr

In [15]:
# Прилагане на функицята за режисьор върху множеството
df['crew'] = df['crew'].apply(get_director_from_crew)
df = df.rename(columns={'crew': 'director'})

df.head(1)

,movie_id,title,overview,genres,keywords,cast,director
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]


In [16]:
# Преобразуване на стойностите на 'overview' колоната от текст във масиц от думи 
df['overview'] = df['overview'].apply(lambda x:x.split())

In [17]:
# Превръщане на всички интервали от избраните колони във празни текстови низове 
columns_to_change = ['cast','director','keywords','genres']
for col in columns_to_change:
    df[col] = df[col].apply(lambda x:[i.replace(' ','') for i in x])

In [18]:
df.head(1)

,movie_id,title,overview,genres,keywords,cast,director
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]


In [19]:
# Обединяване на всички ключови колони в една нова
df['tags'] = df['overview'] + df['genres'] + df['keywords'] + df['cast'] + df['director']

In [20]:
# Премахване на ненужните колони от множеството
df = df[['movie_id', 'title', 'tags']]
df.head(1)

,movie_id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."


In [21]:
# Превръщане на 'tags' колоната от масив във текст
df['tags'] = df['tags'].apply(lambda x:" ".join(x))
df['tags'] = df['tags'].apply(lambda x:x.lower())

In [22]:
df.head(1)

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."


### Нормализиране на 'tags' колоната

In [23]:
# Инициализиране на Porter Stemmer
ps=PorterStemmer()

In [24]:
# Създаване на фукницяата за нормализиране на текст
def stem(text):
    arr=[]
    for i in text.split():
        arr.append(ps.stem(i))
    return " ".join(arr)

In [25]:
# Прилана на фукцията върху 'tags' колоната
df["tags"] = df["tags"].apply(stem)

# Прилагане на CountVectorizer и Косинусова сходимост

In [26]:
# Инициализиране на CountVectorizer 
#(max_features - лимит на речника от думи; stop_words - игнорира частиците от спрецифицирания език)
cv = CountVectorizer(max_features=5000, stop_words="english")

In [27]:
# Прилагане на fit_transform функцията спрямо колоната 'tags'
vector = cv.fit_transform(df["tags"]).toarray()

In [28]:
# Създавана на косинусова матрица на сходимост между филмите
similar = cosine_similarity(vector)

In [29]:
# Създаване на фукцията за препоръчване на филм според подадено заглавие на филк
def recommend(movie):
    movie_index = df[df['title'].apply(lambda x:x.lower()) == movie.lower()].index[0]
    distances = similar[movie_index]
    movie_suggestions = sorted(list(enumerate(distances)), reverse=True,key=lambda x:x[1])[1:6]

    for i in movie_suggestions:
        print(df.iloc[i[0]].title)

# Тестване

In [30]:
recommend('Pirates of the Caribbean: The Curse of the Black Pearl')

Pirates of the Caribbean: Dead Man's Chest
Pirates of the Caribbean: On Stranger Tides
Pirates of the Caribbean: At World's End
VeggieTales: The Pirates Who Don't Do Anything
The Pirates! In an Adventure with Scientists!


In [31]:
recommend('iron man')

Iron Man 3
Iron Man 2
Avengers: Age of Ultron
The Avengers
Captain America: Civil War


In [32]:
recommend('The Godfather')

Desert Dancer
Step Up
Take the Lead
Footloose
Tango
